<a href="https://colab.research.google.com/github/Pavan0704/Column-Transformer/blob/main/Column_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer #for missing values
from sklearn.preprocessing import OneHotEncoder #for categorical values
from sklearn.preprocessing import OrdinalEncoder #for categorical values

In [4]:
df = pd.read_csv('covid toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [7]:
df['has_covid'].value_counts()

,count
has_covid,
No,55
Yes,45


In [8]:
df.isnull().sum() #outputs number of missing values in each column

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)
#above line tells that X has all the columns except 'has_covid' and Y has only 'has_covid'

In [12]:
y_train

,has_covid
90,No
98,No
76,Yes
44,No
19,Yes
...,...
63,No
68,No
73,Yes
89,No


**Without Column Transformer**

In [16]:
#Simple Imputer used to fill the missing values
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape


(80, 1)

In [21]:
# Ordinal Encoding
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

**One Hot Encoding**

In [25]:
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [26]:
#Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [27]:
#Concatenating the columns
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

#also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

**Column Tranformer**

In [28]:
from sklearn.compose import ColumnTransformer

In [34]:
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(),['fever']), #'fever' instead of 'Fever' to match the dataframe column name
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False,drop='first'),['gender','city']) #sparse=False is deprecated, use sparse_output=False instead
],remainder= 'passthrough')

In [35]:
transformer.fit_transform(X_train).shape

(80, 7)

In [36]:
transformer.transform(X_test).shape

(20, 7)